In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import IPython
from IPython.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
# CSS = """
# .output {
#     flex-direction: row;
# }
# """
# HTML('<style>{}</style>'.format(CSS))
# pd.set_option('display.max_columns', None)

In [2]:
main_df = pd.read_csv('./data/0826export.csv')
main_df.columns = main_df.columns.str.lower()

store_master_df = pd.read_excel('./documentation/store_master.xlsx')
store_master_df.columns = store_master_df.columns.str.lower()

---

## Mapping Function:

#### To map from `/store_master.xlsx` onto `main_df`

In [3]:
def map_my_dataframe(main_df, store_master_df, a_list, map_on='store') -> 'pandas.DataFrame':
    """
    > a_list = df2 column names' values(for mapping to df1)
    > rids of outlier comp_pct
    > converts descriptive tidy columns to dtype('category')
    """
    
    for category in a_list:
        df_2_dict = dict(zip(store_master_df[map_on], store_master_df[category]))
        main_df[category] = main_df[map_on].map(df_2_dict)
        
    main_df['store_size'] = pd.cut(main_df.gross_feet, bins=[0, 1000, 2000, 3000, 4000], labels=['sml', 'mdm', 'lrg', 'xlrg'])
    main_df.drop(main_df.comp_pct.values.argmax(), inplace=True) # Large outlier, drop fiscal_week with comp_pct > 200.
    main_df.drop(main_df.comp_pct.values.argmax(), inplace=True) # Large outlierr, drop fiscal_week with comp_pct > 20
    main_df.drop('store_zip', axis=1, inplace=True)
#     main_df.iloc[:, -4:] = main_df.iloc[:, -4:].astype('category') # convert last 4 tidy columns to dtype('category')
    
    return main_df  

df = map_my_dataframe(main_df, store_master_df, store_master_df.columns[2:], map_on='store')
df.head()

,div,store,week_end_date,fiscal_year,fiscal_week,fiscal_month,sales_amt_ty,sales_amt_ly,comp_pct,fiscal_year_ly,...,tot_avg_tenure_days,open_date,close_date,store_state,store_city,gross_feet,class,rptcode,volume_band,store_size
0,7,1504,06-FEB-16,2017,1,1,22003.18,21266.87,0.034622,2016,...,317,01-MAY-12,NaN,NV,LAS VEGAS,1849,Mall,S,2,mdm
1,7,1504,13-FEB-16,2017,2,1,17499.35,23490.64,-0.255050,2016,...,324,01-MAY-12,NaN,NV,LAS VEGAS,1849,Mall,S,2,mdm
2,7,1504,20-FEB-16,2017,3,1,15004.60,22318.80,-0.327715,2016,...,331,01-MAY-12,NaN,NV,LAS VEGAS,1849,Mall,S,2,mdm
3,7,1504,27-FEB-16,2017,4,1,12770.09,17296.40,-0.261691,2016,...,338,01-MAY-12,NaN,NV,LAS VEGAS,1849,Mall,S,2,mdm
4,7,1504,05-MAR-16,2017,5,2,19920.51,24553.82,-0.188700,2016,...,345,01-MAY-12,NaN,NV,LAS VEGAS,1849,Mall,S,2,mdm


---

# Difference comparison:

#### Function compares percentage differences between top 15%, bot 15% of fiscal monthly observations aggregated by mean.

In [20]:
def top_bot_var_compare(df, category, rows) -> "DataFrame[['diff%', 'top', 'bot'], ['diff%', 'top', 'bot'],[...]].sort_values()":
    """
    !!!DOCUMENT HERE!!!
    """
    levels_list = [category, 'store', 'fiscal_year', 'fiscal_month']
    cat_list = sorted(df[category].unique())
    
    df = df.groupby(levels_list).mean()
    dict_df = {}
    
    for cat in cat_list:
        
        cat_df = df.loc[[cat]].iloc[:, 2:].droplevel(levels_list[1:])
#         cat_df.drop(['fiscal_year_ly','fiscal_year_ty', 'fiscal_month', 'fiscal_week'], axis=1)
        
        sample_percent = int(.15 * len(cat_df))
        
        top_df = round(cat_df.nlargest(sample_percent, 'comp_pct').mean(), 3).rename('top_avg')
        bot_df = round(cat_df.nsmallest(sample_percent, 'comp_pct').mean(), 3).rename('bot_avg')
        diff_df = abs(abs(top_df - bot_df) / ((top_df + bot_df) * 0.5) * 100).rename('%diff')
        

        together = pd.concat([diff_df, top_df, bot_df], axis=1)
#         together = diff_df.sort_values()
        
        together = together.sort_values('%diff', ascending=False)
        together.index.names = [f"Vars"]
        together = together.iloc[2:, :].reset_index()
        dict_df[cat] = together.iloc[:rows, :]
    
    
    return pd.concat(dict_df.values(), axis=1, keys=dict_df.keys())

---

## By Class:
#### `['Mall', 'Open Air', 'Downtown', 'Street', 'Airport', 'Lifestyle','Outlet']`

In [22]:
compare_class = top_bot_var_compare(df, 'class', rows=10)
compare_class

Airport                                 \
                       Vars      %diff   top_avg   bot_avg   
0        trans_cnt_exchg_in  76.588949     2.165     0.966   
1         trans_cnt_returns  73.771708     4.611     2.126   
2            am_tenure_days  60.561707  1858.629   994.636   
3        avg_am_tenure_days  57.401262  1414.131   783.420   
4         special_order_amt  57.291118  4009.518  2223.916   
5      cp_total_email_trans  54.978682   519.983   914.242   
6               multi_value  53.599508  4113.656  2374.773   
7            cp_total_trans  53.460052   585.918  1013.422   
8           cp_valid_emails  49.680383   304.174   505.232   
9  cp_valid_mailing_address  49.555524   433.376   718.879   

                       Downtown                                  \
                           Vars       %diff   top_avg   bot_avg   
0                      ft_count  101.176471     0.640     0.210   
1         accessory_other_units   94.212454     2.008     0.722   
2                   multi_value   89.371802  4534.773  1733.665   
3    accessory_shoe_trees_units   88.418115     5.684     2.199   
4    accessory_shoe_trees_value   87.089541   140.521    55.266   
5                   multi_units   85.965080    29.972    11.952   
6            trans_cnt_exchg_in   85.390363     2.769     1.112   
7             special_order_amt   82.981737  3641.305  1505.748   
8         accessory_coats_value   82.128860  1341.850   560.614   
9  accessory_plgs_luggage_value   81.200117   197.477    83.429   

                  Lifestyle             ...  Open Air            \
                       Vars      %diff  ...   top_avg   bot_avg   
0        avg_ft_tenure_days  64.640483  ...   987.155   317.060   
1            ft_tenure_days  64.640483  ...   987.155   317.060   
2            cp_total_trans  56.606975  ...     0.524     0.274   
3           cp_valid_emails  56.561621  ...   284.714   508.381   
4      cp_total_email_trans  55.937859  ...   284.714   508.381   
5  cp_valid_mailing_address  55.123693  ...     0.410     0.233   
6          mark_down_amt_ly  45.144971  ...  2164.877  1241.918   
7       tot_avg_tenure_days  35.282244  ...     0.631     0.381   
8    trans_cnt_empl_returns  33.333333  ...  4142.166  2532.303   
9           tot_tenure_days  32.040140  ...    32.393    20.070   

                      Outlet                                 \
                        Vars      %diff   top_avg   bot_avg   
0             ft_tenure_days  37.552550   347.575   508.271   
1         avg_ft_tenure_days  36.813790   231.365   335.754   
2         trans_cnt_exchg_in  35.302059     3.486     2.440   
3           mark_down_amt_ly  34.016126  2128.566  3001.007   
4      accessory_coats_units  32.410505     8.164     5.887   
5  accessory_shoe_care_value  31.822999   259.771   188.452   
6        trans_cnt_exchg_out  31.231231     0.385     0.281   
7           mark_down_amt_ty  31.140777  3329.554  2432.396   
8                multi_units  30.519475    68.549    50.398   
9  accessory_shoe_care_units  30.306319    42.442    31.272   

                          Street                                 
                            Vars      %diff   top_avg   bot_avg  
0            avg_mgr_tenure_days  81.975589  2306.272   965.319  
1                mgr_tenure_days  81.975589  2306.272   965.319  
2           trans_cnt_empl_sales  81.117570     3.748     1.585  
3         trans_cnt_empl_returns  78.106509     0.235     0.103  
4  accessory_shirt_sweater_units  66.136676    60.928    30.646  
5  accessory_shirt_sweater_value  62.439873  3738.662  1959.652  
6               mark_down_amt_ty  61.549215  2319.049  1227.586  
7             trans_cnt_exchg_in  60.729746     4.216     2.252  
8                    multi_units  58.814534    67.702    36.932  
9          accessory_coats_value  57.234681  1983.981  1101.110  

[10 rows x 28 columns]

---

## By RPT Code:
#### `['S', 'A', 'F']`

In [6]:
compare_rptcode = top_bot_var_compare(df, 'rptcode', rows=10)
compare_rptcode

A                                  \
                A  Top Vars      %diff   top_avg   bot_avg   
0        trans_cnt_exchg_in  76.588949     2.165     0.966   
1         trans_cnt_returns  73.771708     4.611     2.126   
2            am_tenure_days  60.561707  1858.629   994.636   
3        avg_am_tenure_days  57.401262  1414.131   783.420   
4         special_order_amt  57.291118  4009.518  2223.916   
5      cp_total_email_trans  54.978682   519.983   914.242   
6               multi_value  53.599508  4113.656  2374.773   
7            cp_total_trans  53.460052   585.918  1013.422   
8           cp_valid_emails  49.680383   304.174   505.232   
9  cp_valid_mailing_address  49.555524   433.376   718.879   

                          F                                  \
                 F  Top Vars      %diff   top_avg   bot_avg   
0             ft_tenure_days  38.938853   357.139   529.826   
1         avg_ft_tenure_days  38.907805   242.598   359.785   
2           mark_down_amt_ly  35.055982  2134.009  3041.102   
3         trans_cnt_exchg_in  34.654130     3.494     2.462   
4      accessory_coats_units  31.672926     8.174     5.939   
5        trans_cnt_exchg_out  31.137725     0.386     0.282   
6  accessory_shoe_care_value  31.005649   260.512   190.580   
7  accessory_shoe_care_units  29.401050    42.504    31.609   
8   trans_cnt_exchg_original  29.310031     9.447     7.032   
9      accessory_coats_value  28.992341   908.621   678.543   

                           S                                  
                  S  Top Vars      %diff   top_avg   bot_avg  
0             cp_valid_emails  45.000110   421.981   667.003  
1              cp_total_trans  44.755806   794.572  1252.711  
2        cp_total_email_trans  44.457047   673.319  1058.213  
3    cp_valid_mailing_address  44.289395   663.653  1041.184  
4                 multi_value  35.722519  5450.260  3798.343  
5                 multi_units  35.132853    36.194    25.378  
6  accessory_shoe_trees_units  32.091815     5.359     3.877  
7   accessory_shoe_care_value  31.942100   355.246   257.400  
8          trans_cnt_exchg_in  31.380960     3.506     2.555  
9           special_order_amt  31.232885  3320.584  2423.554

---

## By Volume Band:
#### `[1:6]`

In [7]:
compare_vband = top_bot_var_compare(df, 'volume_band', rows=10)
compare_vband

1                                  \
                   1 Top Vars      %diff   top_avg    bot_avg   
0          trans_cnt_exchg_in  52.319588     1.958      1.146   
1  accessory_shoe_trees_units  37.070121     2.654      1.824   
2            mark_down_amt_ly  36.523125  1059.765   1533.299   
3         trans_cnt_exchg_out  36.049383     0.239      0.166   
4       accessory_coats_units  35.712060     5.673      3.954   
5       accessory_coats_value  35.308789   805.702    563.906   
6                 multi_units  34.379423    31.050     21.941   
7            mark_down_amt_ty  34.325641  1612.202   1139.869   
8                sales_amt_ly  33.153558  8917.642  12461.638   
9                 multi_value  32.816215  3199.251   2297.361   

                    2                                   \
           2 Top Vars      %diff    top_avg    bot_avg   
0  avg_ft_tenure_days  95.878910    116.202    330.209   
1      ft_tenure_days  90.201347    147.918    390.952   
2            ft_count  40.791100      0.322      0.487   
3    mark_down_amt_ly  40.273980   1175.762   1768.685   
4        sales_amt_ly  30.557069  11824.788  16089.715   
5         multi_units  29.324880     41.959     31.228   
6      am_tenure_days  28.381373    504.515    671.383   
7  avg_am_tenure_days  27.906291    499.320    661.257   
8         multi_value  27.803002   4638.404   3506.184   
9  trans_cnt_exchg_in  25.273632      3.396      2.634   

                          3             ...          4             \
                 3 Top Vars      %diff  ...    top_avg    bot_avg   
0            cp_total_trans  64.246653  ...   1807.100   3584.174   
1  cp_valid_mailing_address  63.883966  ...  18567.409  26656.179   
2      cp_total_email_trans  63.670295  ...    123.979     86.445   
3           cp_valid_emails  61.884377  ...      0.092      0.130   
4     accessory_other_units  40.035390  ...    692.629    499.548   
5        avg_ft_tenure_days  36.396823  ...      5.362      3.930   
6            ft_tenure_days  36.248128  ...   1413.743   1926.707   
7              sales_amt_ly  30.832187  ...      3.674      2.705   
8     accessory_coats_units  28.707743  ...   2182.609   2949.502   
9               multi_units  27.788799  ...      0.937      1.265   

                          5                                    \
                 5 Top Vars       %diff    top_avg    bot_avg   
0           cp_valid_emails  110.514523    222.560    772.283   
1      cp_total_email_trans  109.659808    361.057   1237.598   
2            cp_total_trans  108.928004    432.222   1466.153   
3  cp_valid_mailing_address  108.755103    357.026   1208.107   
4            am_tenure_days   60.152050   2168.447   4033.852   
5        avg_am_tenure_days   57.958578   1585.515   2879.422   
6          mark_down_amt_ly   45.386836   2323.648   3687.866   
7     accessory_other_value   45.090631    126.803    200.622   
8     accessory_other_units   35.369282      3.221      4.605   
9              sales_amt_ly   35.016586  26539.328  37804.905   

                               6                                   
                      6 Top Vars       %diff    top_avg   bot_avg  
0       cp_valid_mailing_address  174.526374    252.349  3710.165  
1                 cp_total_trans  171.613841    346.278  4533.255  
2                cp_valid_emails  170.592089    181.225  2283.758  
3           cp_total_email_trans  169.828606    310.108  3801.177  
4     accessory_shoe_trees_value  100.257353     55.229   166.257  
5     accessory_shoe_trees_units   99.690118      2.185     6.528  
6  accessory_shirt_sweater_value   50.255674  11256.158  6735.295  
7             avg_am_tenure_days   46.723726   2173.425  3498.490  
8  accessory_shirt_sweater_units   44.236512    159.484   101.712  
9             avg_pt_tenure_days   42.277716    697.566   454.114  

[10 rows x 24 columns]

---

# NEXT: MAKE FUNCTION TO RETURN SINGULAR DATAFRAME TO COMPARE INDEX TOP VARIABLES


In [ ]:
cat_list = df[category].unique()

def one_compare(df,category, rows) -> "DataFrame[['diff%']].sort_values()":
    """
    !!!DOCUMENT HERE!!!
    """
    levels_list = [category, 'store', 'fiscal_year', 'fiscal_month']

    
    df = df.groupby(levels_list).mean()
    dict_df = {}
    
    for cat in cat_list:
        
    cat_df = df.loc[[cat]].iloc[:, 2:].droplevel(levels_list[1:])
        
    sample_percent = int(.15 * len(cat_df))
        
    top_df = round(cat_df.nlargest(sample_percent, 'comp_pct').mean(), 3).rename('top_avg')
    bot_df = round(cat_df.nsmallest(sample_percent, 'comp_pct').mean(), 3).rename('bot_avg')
    diff_df = abs(abs(top_df - bot_df) / ((top_df + bot_df) * 0.5) * 100).rename('%diff')
        
    return diff_df

In [42]:
cat_list = ['Airport', 'Downtown', 'Lifestyle', 'Mall', 'Open Air', 'Outlet', 'Street']

levels_list = ['class', 'store', 'fiscal_year', 'fiscal_month']

df = df.groupby(['class', 'store', 'fiscal_year', 'fiscal_month']).mean()
airport = df.loc[['Airport']].iloc[:, 2:].droplevel(levels_list[1:])



##########################################################################
compare_class.loc[:, ('Mall')].Vars.isin(compare_class.loc[:, ('Airport')].Vars)

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9    False
Name: Vars, dtype: bool

---

## Storing DataFrame as HTML embedded image

In [ ]:
css = """
<style type=\"text/css\">
table {
color: black;
font-family: Helvetica, Arial, sans-serif;
width: 700px;
border-collapse:
collapse; 
border-spacing: 3;
}td, th {
border: 1px solid transparent; /* No more visible border */
height: 40px;
}th {
background: gray; /* Darken header a bit */
font-weight: bold;
}td {
background: lightgray;
text-align: center;
}table tr:nth-child(odd) td{
background-color: white;
}
</style>
"""

import random
import imgkit


def DataFrame_to_image(data, css, outputfile="./out.png", format="png"):
    '''
    For rendering a Pandas DataFrame as an image.
    data: a pandas DataFrame
    css: a string containing rules for styling the output table. This must 
         contain both the opening an closing <style> tags.
    *outputimage: filename for saving of generated image
    *format: output format, as supported by IMGKit. Default is "png"
    '''
    fn = str(random.random()*100000000).split(".")[0] + ".html"
    
    try:
        os.remove(fn)
    except:
        None
    text_file = open(fn, "a")
    
    # write the CSS
    text_file.write(css)
    # write the HTML-ized Pandas DataFrame
    text_file.write(data.to_html())
    text_file.close()
    
    # See IMGKit options for full configuration,
    # e.g. cropping of final image
    imgkitoptions = {"format": format}
    
    imgkit.from_file(fn, outputfile, options=imgkitoptions)
    os.remove(fn)
    
DataFrame_to_image(compare_vband, css, outputfile="./compare_table.png", format="png")